In [1]:
import os
import pickle
import collections

In [2]:
_RAW_COLUMN_NAMES = \
                ['id',          'click',            'hour',         'C1',           'banner_pos',           'site_id',
                 'site_domain', 'site_category',    'app_id',       'app_domain',   'app_category',         'device_id',
                 'device_ip',   'device_model',     'device_type',  'device_conn_type', 'C14',              'C15',
                 'C16',         'C17',              'C18',          'C19',          'C20',                  'C21']
progress_step = 400000
train_filename = '../data/train.csv'
test_filename = '../data/test.csv'

train_dump_filename = 'train_count.pickle'
valid_dump_filename = 'valid_count.pickle'
test_dump_filename = 'test_count.pickle'

output_dir = 'info'
train_dump_path = os.path.join(output_dir, train_dump_filename)
valid_dump_path = os.path.join(output_dir, valid_dump_filename)
test_dump_path = os.path.join(output_dir, test_dump_filename)

In [3]:
with open(train_dump_path, 'rb') as f:
    train_count_dict = pickle.load(f)

with open(valid_dump_path, 'rb') as f:
    valid_count_dict = pickle.load(f)

with open(test_dump_path, 'rb') as f:
    test_count_dict = pickle.load(f)

In [4]:
all_count_dict = { feature_name: collections.defaultdict(lambda:0) for feature_name in _RAW_COLUMN_NAMES[2:]}
useful_count_dict = { feature_name: collections.defaultdict(lambda:0) for feature_name in _RAW_COLUMN_NAMES[2:]}
predict_count_dict = { feature_name: collections.defaultdict(lambda:0) for feature_name in _RAW_COLUMN_NAMES[2:]}
for feature_name in _RAW_COLUMN_NAMES[2:]:
    for feature in train_count_dict[feature_name]:
        all_count_dict[feature_name][feature] += train_count_dict[feature_name][feature]
        useful_count_dict[feature_name][feature] += train_count_dict[feature_name][feature]
    for feature in valid_count_dict[feature_name]:
        all_count_dict[feature_name][feature] += valid_count_dict[feature_name][feature]
        useful_count_dict[feature_name][feature] += valid_count_dict[feature_name][feature]
        predict_count_dict[feature_name][feature] += valid_count_dict[feature_name][feature]
    for feature in test_count_dict[feature_name]:
        all_count_dict[feature_name][feature] += test_count_dict[feature_name][feature]
        predict_count_dict[feature_name][feature] += test_count_dict[feature_name][feature]
for feature_name in all_count_dict:
    all_count_dict[feature_name] = dict(all_count_dict[feature_name])
for feature_name in useful_count_dict:
    useful_count_dict[feature_name] = dict(useful_count_dict[feature_name])
for feature_name in predict_count_dict:
    predict_count_dict[feature_name] = dict(predict_count_dict[feature_name])
    
click_count = train_count_dict['click']['1'] + valid_count_dict['click']['1'], train_count_dict['click']['0'] + valid_count_dict['click']['0']
click_count

(6865066, 33563901)

In [18]:
print('acture unknow feature ...')
unknown_dict_act = dict()
for k,v in test_count_dict.items():
    unknown_list = set(v) - set(useful_count_dict[k])
    if unknown_list:
        unknown_dict_act[k] = { feature:v[feature] for feature in unknown_list}
total_test_record_num = sum(test_count_dict['hour'].values())
list(map(lambda x: (x[0], len(x[1]), sum(x[1].values()), sum(x[1].values())/total_test_record_num), unknown_dict_act.items()))

acture unknow feature ...


[('hour', 24, 4577464, 1.0),
 ('site_id', 105, 818259, 0.17875815080140445),
 ('site_domain', 167, 1404, 0.0003067200528502245),
 ('app_id', 584, 17436, 0.0038090960409519333),
 ('app_domain', 21, 498, 0.00010879386489986595),
 ('device_id', 209565, 383303, 0.08373697750544844),
 ('device_ip', 609169, 1502584, 0.32825686886887584),
 ('device_model', 52, 179, 3.910462212264258e-05),
 ('C14', 259, 1334955, 0.29163637332811354),
 ('C17', 39, 1295908, 0.28310610416597487),
 ('C19', 1, 857, 0.00018722157072125525),
 ('C21', 2, 196768, 0.04298624740686109)]

In [12]:
list(filter(lambda x: x[1] > 1000, unknown_dict_act['site_id'].items()))

[('a7b53e4a', 1749), ('17d1b03f', 813053)]

In [14]:
sorted(unknown_dict_act['site_domain'].items(), key=lambda x: x[1], reverse=True)[:10]

[('c18e2c5f', 365),
 ('57fd8754', 213),
 ('1003ce12', 205),
 ('06fb092e', 140),
 ('5532d220', 115),
 ('0b621dac', 67),
 ('a64fef17', 29),
 ('c233c83f', 27),
 ('31f39ebd', 13),
 ('7310f4eb', 8)]

In [19]:
list(filter(lambda x: x[1] > 1000, unknown_dict_act['app_id'].items()))

[('4bf836e9', 1641),
 ('a1b8be2d', 1141),
 ('24afec8a', 1323),
 ('e6939e63', 1162),
 ('94ad729e', 1261),
 ('8ae6d5d0', 1015),
 ('02aa6429', 2815)]